<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/Self_Driving_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import re
import zipfile

import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds


In [9]:
url = "https://github.com/srihari-humbarwadi/datasets/releases/download/v0.1.0/data.zip"
filename = os.path.join(os.getcwd(), "data.zip")
keras.utils.get_file(filename, url)


with zipfile.ZipFile("data.zip", "r") as z_fp:
    z_fp.extractall("./")


560529408/560525318 [==============================] - 7s 0us/step


In [12]:
# Helper Functions for bounding box:
def swap_x_y(boxes):
  # swaps all the x y values of the boxes
  return tf.stack([ boxes[:,1] ,boxes[:,0], boxes[:,3],boxes[:,2]],axis=-1)

def convert_to_xywh(boxes):
  # Convert [xmin,ymin,xmax,ymax] -> [x,y,width,height]
  return tf.concat(
        [(boxes[..., :2] + boxes[..., 2:]) / 2.0, boxes[..., 2:] - boxes[..., :2]],
        axis=-1,
    )

def convert_to_coners(boxes):
  # Convert [x,y,width,height] -> [xmin,ymin,xmax,ymax]
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0, boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )  


In [14]:
def intersection_over_union(boxes1,boxes2):
  # Calculate the intersection over union between the ground truth and predicted
  # anchor boxes
  boxes1_corners = convert_to_corners(boxes1)
  boxes2_corners = convert_to_corners(boxes2)
  lu = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
  rd = tf.minimum(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])
  intersection = tf.maximum(0.0, rd - lu)
  intersection_area = intersection[:, :, 0] * intersection[:, :, 1]
  boxes1_area = boxes1[:, 2] * boxes1[:, 3]
  boxes2_area = boxes2[:, 2] * boxes2[:, 3]
  union_area = tf.maximum(
        boxes1_area[:, None] + boxes2_area - intersection_area, 1e-8
    )
  # IOU scores between two box sets of shape (N,M)
  return tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)

In [15]:
def visualize_detections(
    image, boxes, classes, scores, figsize=(7, 7), linewidth=1, color=[0, 0, 1]
):
    """Visualize Detections"""
    image = np.array(image, dtype=np.uint8)
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(image)
    ax = plt.gca()
    for box, _cls, score in zip(boxes, classes, scores):
        text = "{}: {:.2f}".format(_cls, score)
        x1, y1, x2, y2 = box
        w, h = x2 - x1, y2 - y1
        patch = plt.Rectangle(
            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
        )
        ax.add_patch(patch)
        ax.text(
            x1,
            y1,
            text,
            bbox={"facecolor": color, "alpha": 0.4},
            clip_box=ax.clipbox,
            clip_on=True,
        )
    plt.show()
    return ax

In [18]:
# Anchor Generator:
  # Fixed sized boxes that the model uses to predict object bounding boxes
  # Regression is used on the offset between the objects center and 
  # the anchor box.
  # Takes the width and height of the anchor box to predict scale of the object.
  # Each location within a feature map 9 anchor boxes, in 3 different scales & 
  # ratios.
class AnchorBox:
  # Generates Anchor Boxes for features maps of strides [8:128] -> at powers of 2
  # Arguments:
  #   Aspect ratio: float list for aspect ratios at each feature map location
  #   scales: float list for scales of anchor boxes at each location in the feature
  #     map.
  #   num_anchors: number of anchors at each location of a feature map
  #   strides: list of strides of each convolutional layer

    def __init__(self):
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]

        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(3, 8)]
        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 256.0, 512.0]]
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        #Computes anchor box dimensions for all ratios and scales at all levels
        # of the feature pyramid.
        
        anchor_dims_all = []
        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]
                )
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))
        return anchor_dims_all

    def _get_anchors(self, feature_height, feature_width, level):
        # Generates anchor boxes for a given feature map size and level
        # Arguments:
        #   feature_height: An integer representing the height of the feature map.
        #   feature_width: An integer representing the width of the feature map.
        #   level: An integer representing the level of the feature map in the
        #     feature pyramid.

        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5
        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * self._strides[level - 3]
        centers = tf.expand_dims(centers, axis=-2)
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        dims = tf.tile(
            self._anchor_dims[level - 3], [feature_height, feature_width, 1, 1]
        )
        anchors = tf.concat([centers, dims], axis=-1)
        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        # Generates anchor boxes for all the feature maps of the feature pyramid.

        # Arguments:
        #   image_height: Height of the input image.
        #   image_width: Width of the input image.


        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(3, 8)
        ]
        return tf.concat(anchors, axis=0)

In [9]:
# Data Preprocessing:

# Horizontal Flip:
def random_flip_horizontal(image,boxes):
  if tf.random.uniform(x) > 0.5:
    image = tf.image.random_flip_left_right(image)
    boxes = tf.stack(
          [1 - boxes[:, 2], boxes[:, 1], 1 - boxes[:, 0], boxes[:, 3]], axis=-1
        )
    return image, boxes

# Resize and Pad Image:
def resize_and_pad_image(image, min_side = 800.0, max_side = 1333.3, 
                         jitter = [640,1024], stride = 128.0):
  # image:     3D tensor [height, width, channels].
  # min_side:  Size to reshape shorter side of image iff jitter == None.
  # max_side:  Size to reshape longer side of image is size is larger than default.
  # jitter:    [min,max] size for scale jittering.
  # stride:     Size of the smallest stride in the  in the feature pyramid.

  image_shape = tf.cast(tf.shape(image)[2:],dtype=tf.float32)

  if jitter is not None:
    min_side = tf.random.uniform((),jitter[0], jitter[1], dtype=tf.float32)
  
  ratio = min_side / tf.reduce_min(image_shape)

  if ratio * tf.reduce_max(image_shape) > max_side:
    ratio = max_side / tf.reduce_max(image_shape)
  
  image_shape = ratio * image_shape
  image = tf.image.resize(iamge, tf.cast(image_shape, dtype=tf.int32))
  padded_image_shape = tf.cast(tf.math.ceil(image_shape/stride)*stride,dtype=tf.int32)

  image = tf.image.pad_to_bounding_box(image, 0, 0,padded_image_shape[0],padded_image_shape[1])

  return image, image_shape, ratio

# Process Data:
def preprocess_data(sample):
    image = sample["image"]
    bbox = swap_xy(sample["objects"]["bbox"])
    class_id = tf.cast(sample["objects"]["label"], dtype=tf.int32)

    image, bbox = random_flip_horizontal(image, bbox)
    image, image_shape, _ = resize_and_pad_image(image)

    bbox = tf.stack(
        [
            bbox[:, 0] * image_shape[1],
            bbox[:, 1] * image_shape[0],
            bbox[:, 2] * image_shape[1],
            bbox[:, 3] * image_shape[0],
        ],
        axis=-1,
    )
    bbox = convert_to_xywh(bbox)
    return image, bbox, class_id

In [15]:
# Label Encoding:
class LabelEncoder:
  # turns raw labels from data to targets suitable for training
  def __init__(self):
    self.anchor_box = AnchorBox()
    self._box_variance = tf.convert_to_tensor(
        [0.1,0.1,0.2,0.2], dtype = tf.float32
    )

  def _match_anchor_boxes(self, anchor_boxes, gt_boxes, match_iou=0.5,ignore_iou=0.4):
    # Uses Intercetion Over Union (IOU) to match Ground Truth (gt) boxes with Anchor boxes
    # Steps:
        # 1 -> Find pairwise IOU for each 'M' Anchor boxes and 'N' Ground Truth boxes and
              # store in an [M,N] matrix.
        # 2 -> If IOU > match_iou then the ground truth box with the maximum IOU is 
              # assigned to anchor box of each rox.
        # 3 -> If max IOU in each row < ingore_iou then anchor box is set to background.
        # 4 -> Any boxes not assigned a box is ingnored in training. 
    # Returns:
      # matched_gt_idx:  Index of matched object.
      # positive_mask:   Mask for anchor boxes set assigned to ground truth boxes.
      # ignore_mask:     A mask for anchor boxes that need to be ignored.

      # Step 1:
      iou_matrix = compute_iou(anchor_boxes,gt_boxes)
      # Step 2:
      max_iou = tf.reduce_max(iou_matrix,axis=1)
      matched_gt_idx = tf.argmax(iou_matrix,axis=1)
      positive_mask = tf.greater_equal(max_iou,match_iou)
      # Step 3:
      negative_mask = tf.less(max_iou, ignore_iou)
      # Step 4:
      ignore_mask = tf.logical_not(tf.logical_or(positive_mask, negative_mask))
      
      return (
            matched_gt_idx,
            tf.cast(positive_mask, dtype=tf.float32),
            tf.cast(ignore_mask, dtype=tf.float32),
        )
  
  def _compute_box_target(self,anchor_boxes,matched_gt_boxes):
    # Convert Ground Truth Boxes to labels for training:
    box_target = tf.concat(
    [
        (matched_gt_boxes[:, :2] - anchor_boxes[:, :2]) / anchor_boxes[:, 2:],
            tf.math.log(matched_gt_boxes[:, 2:] / anchor_boxes[:, 2:]),],
            axis=-1,)
    box_target = box_target / self._box_variance
    return box_target

  def _encode_sample(self, image_shape, gt_boxes, class_ids):
    # Creates bounding box and classification lables:
    anchor_boxes = self._anchor_box.get_anchors(image_shape[1], image_shape[2])
    cls_ids = tf.cast(cls_ids, dtype=tf.float32)
    matched_gt_idx, positive_mask, ignore_mask = self._match_anchor_boxes(
            anchor_boxes, gt_boxes
        )
    matched_gt_boxes = tf.gather(gt_boxes, matched_gt_idx)
    box_target = self._compute_box_target(anchor_boxes, matched_gt_boxes)
    matched_gt_cls_ids = tf.gather(cls_ids, matched_gt_idx)
    cls_target = tf.where(
            tf.not_equal(positive_mask, 1.0), -1.0, matched_gt_cls_ids
        )
    cls_target = tf.where(tf.equal(ignore_mask, 1.0), -2.0, cls_target)
    cls_target = tf.expand_dims(cls_target, axis=-1)
    label = tf.concat([box_target, cls_target], axis=-1)
    return label    

  def encode_batch(self, batch_images, gt_boxes, class_ids):
    # Creates bounding boxes and classification labels for a whole batch.
    images_shape = tf.shape(batch_images)
    batch_size = images_shape[0]
    labels = tf.TensorArray(dtype=tf.float32, size=batch_size, dynamic_size=True)
    
    for i in range(batch_size):
        label = self._encode_sample(images_shape, gt_boxes[i], cls_ids[i])
        labels = labels.write(i, label)
    batch_images = tf.keras.applications.resnet.preprocess_input(batch_images)
    return batch_images, labels.stack()   